#### 0. 필수 package 설치
- dlib 설치
  - Windows 의 경우 아래 주소에서 python version에 맞는 whl 파일 download 받아서 설치
  - https://github.com/z-mahmud22/Dlib_Windows_Python3.x/tree/main
  - `pip install dlib-19.22.99-cp310-cp310-win_amd64.whl`

- 기타 library 설치

In [2]:
!pip install faiss-cpu face_recognition sqlalchemy numpy

  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-win_amd64.whl.metadata (3.8 kB)
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached faiss_cpu-1.8.0.post1-cp310-cp310-win_amd64.whl (14.6 MB)
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 1. 데이터베이스 설정 (SQLAlchemy)

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Face(Base):
    __tablename__ = 'faces'
    
    id = Column(Integer, primary_key=True)
    image_file = Column(String)  # 이미지 파일명
    face_index = Column(Integer)  # 얼굴 인덱스 (한 이미지에 여러 얼굴이 있을 수 있음)
    encoding = Column(LargeBinary)  # 얼굴 인코딩 (numpy 배열을 binary로 저장)
    top = Column(Integer)  # 얼굴 위치 (top 좌표)
    right = Column(Integer)
    bottom = Column(Integer)
    left = Column(Integer)

In [2]:
# SQLite 엔진 생성 및 세션 설정
engine = create_engine('sqlite:///faces.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

#### 2. 얼굴 인코딩 저장

In [3]:
import face_recognition
import os
import numpy as np

# 얼굴 DB 구축 및 SQLite에 저장
def build_face_db(image_dir):
    for image_file in os.listdir(image_dir):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_dir, image_file)
            image = face_recognition.load_image_file(image_path)
            encodings = face_recognition.face_encodings(image)
            locations = face_recognition.face_locations(image)

            if encodings:
                for i, (encoding, location) in enumerate(zip(encodings, locations)):
                    face = Face(
                        image_file=image_file,
                        face_index=i,
                        encoding=np.array(encoding).tobytes(),  # numpy 배열을 binary로 저장
                        top=location[0],
                        right=location[1],
                        bottom=location[2],
                        left=location[3]
                    )
                    session.add(face)
    
    session.commit()

In [4]:
# 얼굴 DB 구축 실행
faces_path = './faces'
build_face_db(faces_path)

#### 3. Faiss 인덱스 생성 및 저장

In [5]:
import faiss

# Faiss 인덱스와 ID 매핑을 파일로 저장하는 함수
def save_faiss_index(index, id_map, index_file='faiss_index.index', id_map_file='id_map.npy'):
    faiss.write_index(index, index_file)
    np.save(id_map_file, id_map)

# Faiss 인덱스와 ID 매핑을 파일에서 불러오는 함수
def load_faiss_index(index_file='faiss_index.index', id_map_file='id_map.npy'):
    index = faiss.read_index(index_file)
    id_map = np.load(id_map_file)
    return index, id_map

# Faiss 인덱스 생성 및 저장
def build_and_save_faiss_index():
    # SQLite에서 얼굴 인코딩 가져오기
    faces = session.query(Face.id, Face.encoding).all()

    vector_dim = 128  # face_recognition 얼굴 인코딩 차원
    index = faiss.IndexFlatL2(vector_dim)

    encodings = []
    face_ids = []
    for face in faces:
        encoding = np.frombuffer(face.encoding, dtype=np.float64)
        encodings.append(encoding)
        face_ids.append(face.id)

    if encodings:
        encodings_np = np.array(encodings).astype(np.float32)
        index.add(encodings_np)

    # ID 매핑 배열 생성
    id_map = np.array(face_ids)

    # 인덱스와 ID 매핑 저장
    save_faiss_index(index, id_map)

    return index, id_map

In [6]:
# 인덱스 구축 후 파일로 저장
build_and_save_faiss_index()

(<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001FA8425AAC0> >,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40]))

#### 4. 유사한 얼굴 찾기 함수 정의(Faiss 인덱스 사용)

In [7]:
from sqlalchemy import select

def find_similar_faces_with_faiss(query_image_path, faiss_index, id_map, k=5):
    query_image = face_recognition.load_image_file(query_image_path)
    query_encodings = face_recognition.face_encodings(query_image)

    if not query_encodings:
        print("이미지에서 얼굴을 찾을 수 없습니다.")
        return []

    similar_faces = []
    for query_encoding in query_encodings:
        query_encoding = np.array(query_encoding).reshape(1, -1).astype(np.float32)

        distances, indices = faiss_index.search(query_encoding, k)

        face_ids = id_map[indices[0]]
        
        # Face.id와 거리를 딕셔너리로 매핑
        id_distance_map = dict(zip(face_ids, distances[0]))

        # 데이터베이스에서 Face 객체 조회
        stmt = select(Face).where(Face.id.in_(face_ids.tolist()))
        result = session.execute(stmt).scalars().all()

        # 각 Face 객체에 대해 정확한 거리 값을 찾아 리스트에 추가
        for face in result:
            distance = id_distance_map[face.id]
            similar_faces.append((face, distance))

        # 거리에 따라 결과 정렬
        similar_faces.sort(key=lambda x: x[1])

    return similar_faces

#### 5. 시각화 함수 정의

In [8]:
from PIL import Image, ImageDraw, ImageFont

# 매칭된 이미지의 얼굴을 박스 처리하고 유사도 출력하기
def draw_faces(matched_faces, faces_path):
    for face, distance in matched_faces:
        image_path = os.path.join(faces_path, face.image_file)
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # 얼굴 위치 가져오기
        top, right, bottom, left = face.top, face.right, face.bottom, face.left
        
        # 얼굴 주위에 박스 그리기
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 255, 0), width=3)

        # 유사도 값 소수점 두 자리로 변환하여 문자열로 저장
        similarity_text = f"{distance:.2f}"

        # 얼굴 박스 위에 유사도 값 표시 (박스의 좌측 상단에 표시)
        text_position = (left, top - 40)  # 텍스트 위치를 박스 위로 조정
        
        # 폰트 설정 (폰트 크기를 크게 조정)
        font = ImageFont.truetype("arial.ttf", 24)  # 폰트 경로와 크기 설정
        
        # 글자 테두리 그리기 (흰색 테두리로 어두운 배경에서도 가독성 향상)
        # 테두리를 먼저 흰색으로 그린 후, 중앙에 검은 글자를 그리는 방식
        outline_color = "white"
        draw.text((text_position[0] - 1, text_position[1] - 1), similarity_text, font=font, fill=outline_color)
        draw.text((text_position[0] + 1, text_position[1] - 1), similarity_text, font=font, fill=outline_color)
        draw.text((text_position[0] - 1, text_position[1] + 1), similarity_text, font=font, fill=outline_color)
        draw.text((text_position[0] + 1, text_position[1] + 1), similarity_text, font=font, fill=outline_color)

        # 본문 텍스트 (검은 글자)
        draw.text(text_position, similarity_text, font=font, fill="black")

        # 매칭된 얼굴 이미지 보여주기
        image.show()

#### 6. 실행

In [9]:
# 쿼리 이미지 설정
query_image_path = './test_hanni.jpg'

In [10]:
# Faiss Load
faiss_index, id_map = load_faiss_index()

# 쿼리 이미지로 비슷한 얼굴 찾기
similar_faces = find_similar_faces_with_faiss(query_image_path, faiss_index, id_map, k=5)

if similar_faces:
    print("비슷한 얼굴들:")
    for face, distance in similar_faces:
        print(f"{face.image_file} 얼굴 {face.face_index} (유사도 거리: {distance})")
    
    # 매칭된 이미지에서 얼굴을 그리기
    draw_faces(similar_faces, faces_path)
else:
    print("유사한 얼굴을 찾을 수 없습니다.")

비슷한 얼굴들:
newjeans02.jpg 얼굴 4 (유사도 거리: 0.06112991273403168)
newjeans01.jpg 얼굴 0 (유사도 거리: 0.09109141677618027)
hanni02.jpg 얼굴 0 (유사도 거리: 0.1049562394618988)
hanni06.jpg 얼굴 0 (유사도 거리: 0.1453244686126709)
newjeans02.jpg 얼굴 0 (유사도 거리: 0.154688760638237)
newjeans01.jpg 얼굴 2 (유사도 거리: 0.17067113518714905)
newjeans02.jpg 얼굴 2 (유사도 거리: 0.18196284770965576)
hanni04.jpg 얼굴 0 (유사도 거리: 0.1861667037010193)
nobody.jpg 얼굴 2 (유사도 거리: 0.19330571591854095)
hanni03.jpg 얼굴 0 (유사도 거리: 0.20511700212955475)
신세경.jpg 얼굴 0 (유사도 거리: 0.2137187123298645)
newjeans01.jpg 얼굴 3 (유사도 거리: 0.21556031703948975)
newjeans02.jpg 얼굴 1 (유사도 거리: 0.21713560819625854)
newjeans01.jpg 얼굴 1 (유사도 거리: 0.22216691076755524)
김희선.jpg 얼굴 0 (유사도 거리: 0.22664082050323486)
hanni01.jpg 얼굴 0 (유사도 거리: 0.23257119953632355)
nobody.jpg 얼굴 0 (유사도 거리: 0.23509380221366882)
김태희.jpg 얼굴 0 (유사도 거리: 0.2478526532649994)
newjeans01.jpg 얼굴 4 (유사도 거리: 0.2622310519218445)
nobody.jpg 얼굴 3 (유사도 거리: 0.2630600929260254)
